In [1]:

%load_ext autoreload
%autoreload 2

In [1]:
from pytorch_lightning.cli import LightningCLI

## Testing SmilesDataset

In [1]:
from smiles_dataset import SmilesDataset
from smiles_lightning_data_module import SmilesDataModule
from lightning_model import LightningClassicGNN
import pytorch_lightning as pl
from torch_geometric.transforms import distance
from torch_geometric.loader import DataLoader
import os
import torch
# making sure we are as determinstic as possibe
torch.use_deterministic_algorithms(True)
import numpy as np

from torch_geometric.data import Data
from typing import List, Callable
from functools import partial
from smiles_dataset import SmilesDataset
from torch_geometric.transforms import Compose, distance
from datasets import BaceDataset

%load_ext autoreload
%autoreload 2

def filter_target(target_names:List[str], target:str)-> Callable[[Data],Data]:
    """ Transform to be given to SmilesDataset, has the effect of filtering out all irelevant targets in the Data objects in the dataset at runtime
    Example: for BACE, target_names=['Class', 'PIC50'], we want to train a classifier => target='Class'
    """
    target_idx = target_names.index(target)

    return partial(filter_target_with_idx, target_idx=target_idx)

def filter_target_with_idx(graph:Data, target_idx:int) -> Data:
    new_graph = Data(x=graph.x, edge_index=graph.edge_index, edge_attr=graph.edge_attr, pos=graph.pos ,y=graph.y[:,target_idx:target_idx+1], z=graph.z, name=graph.name, idx=graph.idx) 

    return new_graph


root= "../data/bace"
filename="bace.csv"
seed=42
## pytorch lighting takes of seeding everything
pl.seed_everything(seed=seed, workers=True)


#! rm -rf ../data/bace/processed

data,target_names = BaceDataset(root=root)
del data
transforms=Compose([filter_target(target_names=target_names, target='Class'), distance.Distance()])
dataset = SmilesDataset(root=root,filename="bace.csv", transform=transforms)

# from torch dataset, create lightning data module to make sure training splits are always done the same ways
data_module = SmilesDataModule(dataset=dataset, seed=seed)

num_node_features = data_module.num_node_features
num_edge_features= data_module.num_edge_features

gnn_model = LightningClassicGNN(classification=True, num_classes=2, dropout_p=0.5,num_hidden_features=32,  num_node_features=num_node_features, num_edge_features=num_edge_features)

num_epochs=1


# from pytorch_ligthing import loggers
# logger = loggers.WandbLogger()

# default root dir is where the logs and weights are logged
# useful when debugging is limit_train_batches
# by default uses TensorBoardLogger, can be configured 
# Plugins allows us to connect to arbitrary cluster
# can set max_epochs
# can use precision to specify number of bit floating points to reduce memory footprint ()
# can use accumulate_grad_batches to speed-up training too
trainer = pl.Trainer(deterministic=True, default_root_dir=os.getcwd(), precision="bf16", max_epochs=num_epochs)



Global seed set to 42
Global seed set to 42
Using bfloat16 Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/harold/miniconda3/envs/chem/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1764: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


In [48]:
# tune to find the learning rate
trainer.tune(gnn_model,datamodule=data_module)

In [11]:

# we can resume from a checkpoint using trainer.fit(ckpth_path="some/path/to/my_checkpoint.ckpt")
trainer.fit(gnn_model, datamodule=data_module)


  | Name  | Type              | Params
--------------------------------------------
0 | model | Sequential_01044e | 4.9 K 
--------------------------------------------
4.9 K     Trainable params
0         Non-trainable params
4.9 K     Total params
0.020     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

x_out torch.BFloat16Tensor
batch torch.DoubleTensor
x_out torch.BFloat16Tensor
batch torch.DoubleTensor


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

x_out torch.BFloat16Tensor
batch torch.DoubleTensor
x_out torch.BFloat16Tensor
batch torch.DoubleTensor
x_out torch.BFloat16Tensor
batch torch.DoubleTensor


/home/harold/miniconda3/envs/chem/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:98: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1078. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


x_out torch.BFloat16Tensor
batch torch.DoubleTensor


/home/harold/miniconda3/envs/chem/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:98: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1036. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
/home/harold/miniconda3/envs/chem/lib/python3.9/site-packages/pytorch_lightning/utilities/data.py:98: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 811. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
`Trainer.fit` stopped: `max_epochs=1` reached.


x_out torch.BFloat16Tensor
batch torch.DoubleTensor


In [12]:
trainer.validate(gnn_model, datamodule=data_module)

Validation: 0it [00:00, ?it/s]

batch_size 32
batch_size 32
batch_size 32
batch_size 32
batch_size 23
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     accuracy/valid         15.602648735046387
       loss/valid              96459.4453125
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'loss/valid': 96459.4453125, 'accuracy/valid': 15.602648735046387}]